In [2]:
import pandas as pd
from collections import defaultdict
csv_file = "MBTA-Bus-Arrival-Departure-Times_2024-01.csv"

In [ ]:
df = pd.read_csv(csv_file, nrows=100000)

# Check what data looks like
# print(df.head())

# Check the types
# print(df.dtypes)

df["service_date"] = pd.to_datetime(df["service_date"])

df["scheduled"] = pd.to_datetime(df["scheduled"]).dt.strftime("%H:%M:%S")
df["actual"] = pd.to_datetime(df["actual"]).dt.strftime("%H:%M:%S")

df["scheduled"] = pd.to_datetime(df["service_date"].dt.date.astype(str) + " " + df["scheduled"])
df["actual"] = pd.to_datetime(df["service_date"].dt.date.astype(str) + " " + df["actual"])

# We ignore earliness as there is no indication of what it means and how it's calculated 
# in the source (i.e. being late gives positive earliness scores sometimes but usually negative)
df = df.drop(columns=["service_date"])

# calculate our own lateness score which is just in seconds
df["lateness"] = (df["actual"] - df["scheduled"]).dt.total_seconds()
average_lateness = df["lateness"].mean()

lateness_per_route = df.groupby("route_id")["lateness"].mean()

lateness_per_route


In [ ]:
# Loading entire csv into memory crashes computer so we process in chunks
chunk_size = 500_000

for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
    print(chunk.shape)

(500000, 14)
(500000, 14)
(500000, 14)


KeyboardInterrupt: 